# Exploring Ray API Calls

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_07_ray_data.ipynb) <br>
⬅️ [Previous notebook](./ex_05_multiprocess_pool.ipynb) <br>

### Overview
Ray offers a rich and wide set of APIs across all its components. Discussing or covering all of them is out of the scope of this notebook (and tutorial), albeit we will touch upon some common APIs, its use, and arguments. A more exhaustive references below in the documentation provides a full set.

 * [Ray Core](https://docs.ray.io/en/latest/ray-core/package-ref.html)
 * [Ray AIR](https://docs.ray.io/en/latest/ray-air/package-ref.html)
 * [Ray Data](https://docs.ray.io/en/latest/data/dataset.html)
 * [Ray Train](https://docs.ray.io/en/latest/train/train.html)
 * [Ray Tune](https://docs.ray.io/en/latest/tune/index.html)
 * [Ray Serve](https://docs.ray.io/en/latest/serve/index.html)
 * [RLlib](https://docs.ray.io/en/latest/rllib/index.html)


### Learning objectives
In this quick tour of the API, you will learn about:

 * Common Ray Core APIs
 * Some useful arguments to these APIs 
 * Tips and Tricks for first-time users


This lesson explores a few of the other API calls you might find useful, as well as options that can be used with the API calls we've already used in the previous lessons. Additionally, we will walk through some tips and tricks for first time users.

> **Tip:** The [Ray Package Reference](https://docs.ray.io/en/latest/package-ref.html) in the [Ray Docs](https://docs.ray.io/en/latest/) is useful for exploring the API features we'll learn.

In [4]:
import time 
import sys
import logging 
import random
import math
import json
import requests
import numpy as np
from typing import List, Tuple

import ray

## ray.init()

When we used [`ray.init()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.init), we used it to start Ray on our local machine. When the optional `address=...` argument is specified, the driver connects to the corresponding Ray cluster.

There are a lot of optional keyword arguments you can pass to `ray.init()`. Here are some of them. All options are described in the [documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.init). 

| Name | Type | Example | Description |
| :--- | :--- | :------ | :---------- |
| `address` | `str` | `address='auto'` | The address of the Ray cluster to connect to. If this address is not provided, then this command will start a raylet, a plasma store, a plasma manager, and some workers. It will also kill these processes when Python exits. If the driver is running on a node in a Ray cluster, using `auto` as the value tells the driver to detect the the cluster, removing the need to specify a specific node address. |
| `num_cpus` | `int` | `num_cpus=4` | Number of CPUs the user wishes to assign to each _raylet_. |
| `num_gpus` | `int` | `num_gpus=1` | Number of GPUs the user wishes to assign to each _raylet_. |
| `resources` | `dictionary` | `resources={'resource1': 4, 'resource2': 16}` | Maps the names of custom resources to the quantities of those resources available. |
| `memory` | `int` | `memory=1000000000` | The amount of memory (in bytes) that is available for use by workers requesting memory resources. By default, this is automatically set based on the available system memory. |
| `object_store_memory` | `int` | `object_store_memory=1000000000` | The amount of memory (in bytes) for the object store. By default, this is automatically set based on available system memory, subject to a 20GB cap. |
| `log_to_driver` | `bool` | `log_to_driver=True` | If true, then the output from all of the worker processes on all nodes will be directed to the driver program. |
| `local_mode` | `bool` | `local_mode=True` | If true, the code will be executed serially. This is useful for debugging. |
| `ignore_reinit_error` | `bool` | `ignore_reinit_error=True` | If true, Ray suppresses errors from calling `ray.init()` a second time (as we've done in these notebooks). Ray won't be restarted. |
| `configure_logging` | `bool` | `configure_logging=True` | If true (default), configuration of logging is allowed here. Otherwise, the user may want to configure it separately. |
| `logging_level` | _Flag_ | `logging_level=logging.INFO` | The logging level, defaults to `logging.INFO`. Ignored unless "configure_logging" is true. |
| `logging_format` | `str` | `logging_format='...'` | The logging format to use, defaults to a string containing a timestamp, filename, line number, and message. See the Ray source code `ray_constants.py` for details. Ignored unless "configure_logging" is true. |
| `runtime_env` | `map` | `{"working_dir": "/path/to/files"}` | Your Ray application might depend on source files or data files. For a development workflow, these might live on your local machine, but when it comes time to run things at scale, you will need to get them to your remote cluster. A way to send these files across all nodes in the cluster so that your distributed tasks or actors can access them, use this option, [for example.](https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#runtime-environments) |

Let's define a global runtime enviornment accessible or available across all nodes by all Ray tasks and actors.
 * we want our working directory with some data files available across these tasks and actors. There is a limit of 2GB upload
 * we additional packages available too; they will be installed automatically upon start or launch of the cluster
 
A runtime environment describes the dependencies your Ray application needs to run, including files, packages, environment variables, and more. It is installed dynamically on the cluster at runtime and cached for future use. A short [blog](https://medium.com/@2twitme/day-10-how-to-use-ray-runtime-environment-dependencies-5d6266c48c9) describing how to use Runtime environments. 

In [5]:
ENV_VARS = {"BUCKET_LOC": "s3://models",
            "MODEL_NAME": "lr_model.pkl"
           }
MY_RUNTIME_ENV = {"working_dir": "data", 
                   "env_vars": ENV_VARS, 
                   "pip": ["requests", "pendulum==2.1.2"]
                 }

In [6]:
if ray.is_initialized:
    ray.shutdown()
ray.init(num_cpus=4, runtime_env=MY_RUNTIME_ENV,logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8265


See also the documentation for [ray.shutdown()](https://ray.readthedocs.io/en/latest/package-ref.html#ray.shutdown), which is needed in some contexts.

## Fetching Cluster Information

Many methods return information:

| Method | Brief Description |
| :----- | :---------------- |
| [`ray.get_gpu_ids()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get_gpu_ids) | GPUs |
| [`ray.nodes()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.nodes) | Cluster nodes |
| [`ray.cluster_resources()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.cluster_resources) | All the available resources, used or not |
| [`ray.available_resources()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.available_resources) | Resources not in use |

You can see the full list of methods in the [ray.init()](https://docs.ray.io/en/latest/ray-core/package-ref.html#python-api) API documention.

In [4]:
print(f"""
ray.get_gpu_ids():          {ray.get_gpu_ids()}
ray.nodes():                {ray.nodes()}
ray.cluster_resources():    {ray.cluster_resources()}
ray.available_resources():  {ray.available_resources()}
""")


ray.get_gpu_ids():          []
ray.nodes():                [{'NodeID': '9f654156614fbb90c7c440a0a9ed45f18e5dc329329c0a51d6c55c5b', 'Alive': True, 'NodeManagerAddress': '127.0.0.1', 'NodeManagerHostname': 'Juless-MacBook-Pro-16', 'NodeManagerPort': 57166, 'ObjectManagerPort': 57165, 'ObjectStoreSocketName': '/tmp/ray/session_2022-12-31_07-19-39_860216_4839/sockets/plasma_store', 'RayletSocketName': '/tmp/ray/session_2022-12-31_07-19-39_860216_4839/sockets/raylet', 'MetricsExportPort': 61821, 'NodeName': '127.0.0.1', 'alive': True, 'Resources': {'memory': 50165835367.0, 'CPU': 4.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 2147483648.0}}]
ray.cluster_resources():    {'object_store_memory': 2147483648.0, 'memory': 50165835367.0, 'CPU': 4.0, 'node:127.0.0.1': 1.0}
ray.available_resources():  {'CPU': 4.0, 'object_store_memory': 2147483648.0, 'node:127.0.0.1': 1.0, 'memory': 50165835367.0}



Recall that we used `ray.init(num_cpus=4...)` above to initialize so `ray.nodes()[0]['Resources']['CPU']` returns number of CPU cores on our machines:

In [5]:
ray.nodes()[0]['Resources']['CPU']

4.0

## @ray.remote()

We've used [@ray.remote](https://ray.readthedocs.io/en/latest/package-ref.html#ray.remote) a lot. You can pass arguments when using it. Here are some of them.

| Name | Type | Example | Description |
| :--- | :--- | :------ | :---------- |
| `num_cpus` | `int` | `num_cpus=4` | The number of CPU cores to reserve for this task or for the lifetime of the actor. |
| `num_gpus` | `int` | `num_gpus=1` | The number of GPU cores to reserve for this task or for the lifetime of the actor. |
| `num_returns` | `int` | `num_returns=2` | (Only for tasks, not actors.) The number of object refs returned by the remote function invocation. |
| `runtime_env` | `map` | `runtime_env = {"working_dir": ".", "pip": ["requests"]}}` | The runtime environment to use for this job (see [Runtime environments](https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#runtime-environments) for details. |
| `max_calls` | `int` | `max_calls=5` | Only for *remote tasks*. This specifies the maximum of times that a given worker can execute the given remote function before it must exit (this can be used to address memory leaks in third-party libraries or to reclaim resources that cannot easily be released, e.g., GPU memory that was acquired by TensorFlow). By default this is infinite. |
| `max_restarts` | `int` | `max_restarts=-1` | Only for *actors*. This specifies the maximum number of times that the actor should be restarted when it dies unexpectedly. The minimum valid value is 0 (default), which indicates that the actor doesn't need to be restarted. A value of -1 indicates that an actor should be restarted indefinitely. |
| `max_task_retries` | `int` | `max_task_retries=-1` | Only for *actors*. How many times to retry an actor task if the task fails due to a system error, e.g., the actor has died. If set to -1, the system will retry the failed task until the task succeeds, or the actor has reached its max_restarts limit. If set to n > 0, the system will retry the failed task up to n times, after which the task will throw a `RayActorError` exception upon `ray.get`. Note that Python exceptions are not considered system errors and will not trigger retries. |
| `max_retries` | `int` | `max_retries=-1` | Only for *remote functions*. This specifies the maximum number of times that the remote function should be rerun when the worker process executing it crashes unexpectedly. The minimum valid value is 0, the default is 4 (default), and a value of -1 indicates infinite retries. |

Here's an example with and without `num_return_vals`:

### @ray.method()

Related to `@ray.remote()`, [@ray.method()](https://ray.readthedocs.io/en/latest/package-ref.html#ray.method) allows you to specify the number of return values for a method in a task or an actor, by passing the `num_returns` keyword argument. None of the other `@ray.remote()` keyword arguments are allowed. Here is an example:

In [6]:
@ray.remote(num_cpus=1, num_returns=3)
def tuple3(id: str, two:int, lst: List[float]) -> Tuple[str, int, float]:
    one = id.upper()
    two = random.randint(5, 10)
    three = sum(lst)
    return (one, two, three)

# Return three object references with three distinct values in each 
x_ref, y_ref, z_ref = tuple3.remote("lionel messie", 1, [2.2, 4.4, 5.5])

# Fetch the entire list
x, y, z = ray.get([x_ref, y_ref, z_ref])
print(f'({x}, {y}, {z})')

(LIONEL MESSIE, 9, 12.100000000000001)


A slight variation of the above example is pack all values in a single return, and then unpack them.

In [7]:
@ray.remote(num_returns=1)
def tuple3_packed(id: str, two:int, lst: List[float]) -> Tuple[str, int, float]:
    one = id.upper()
    two = random.randint(5, 10)
    three = sum(lst)
    return (one, two, three)

# Returns one object references with three values in it
xyz_ref = tuple3_packed.remote("lionel messie", 1, [2.2, 4.4, 5.5])

# Fetch from a single object ref and unpack into three values
x, y, z = ray.get(xyz_ref)
print(f'({x}, {y}, {z})')

(LIONEL MESSIE, 8, 12.100000000000001)


Let's do the same for an Ray actor method

In [8]:
@ray.remote
class TupleActor:
    @ray.method(num_returns=3)
    def tuple3(self, id: str, two:int, lst: List[float]) -> Tuple[str, int, float]:
        one = id.upper()
        two = random.randint(5, 10)
        three = sum(lst)
        return (one, two, three)
    
# Create an instance of an actor
actor = TupleActor.remote()
x_ref, y_ref, z_ref = actor.tuple3.remote("lionel messie", 1, [2.2, 4.4, 5.5])
x, y, z = ray.get([x_ref, y_ref, z_ref])
print(f'({x}, {y}, {z})')   

(LIONEL MESSIE, 5, 12.100000000000001)


# Tips and Tricks for first-time users
First time users can trip upon certain API calls in Ray's usage patterns. This short tips & tricks will insure you against unexpected results. Below we briefly explore a handful of API calls and their best practice.

### Tip 1: Delay ray.get()

With Ray, all invocations of `.remote()` calls are asynchronous, meaning the operation  returns immediately with a promise/future object Reference ID. This is key to achieving massive parallelism, for it allows a devloper to launch many remote tasks, each returning a remote future object ID. Whenever needed, this object ID is fetched with `ray.get`. Because `ray.get` is a blocking call, where and how often you use can affect the performance of your Ray application. 


In [9]:
@ray.remote
def do_some_work(x):
    # Assume doing some computation
    time.sleep(0.5)
    return math.exp(x)

#### Bad usage
We use `ray.get` inside a list comprehension loop, hence it blocks on each call of `.remote()`, delaying until the task is finished and the value
is materialized and fetched from the Ray object store.

In [10]:
%%time
results = [ray.get(do_some_work.remote(x)) for x in range(10)]
results

CPU times: user 49.9 ms, sys: 23.9 ms, total: 73.9 ms
Wall time: 5.1 s


[1.0,
 2.718281828459045,
 7.38905609893065,
 20.085536923187668,
 54.598150033144236,
 148.4131591025766,
 403.4287934927351,
 1096.6331584284585,
 2980.9579870417283,
 8103.083927575384]

#### Good usage
We delay `ray.get` after all the tasks have been invoked and their references have been returned. That is,
we don't block on each call but instead do outside the comprehension loop.


In [11]:
%%time
results = ray.get([do_some_work.remote(x) for x in range(10)])
results

CPU times: user 19.1 ms, sys: 11.7 ms, total: 30.8 ms
Wall time: 2.24 s


[1.0,
 2.718281828459045,
 7.38905609893065,
 20.085536923187668,
 54.598150033144236,
 148.4131591025766,
 403.4287934927351,
 1096.6331584284585,
 2980.9579870417283,
 8103.083927575384]

#### Takeway tip 1: 
Since `ray.get` is a blocking call, postpone its use only when you need object ID's value. If called eagerly, it can
affect the performance of your desired parallelism.

### Tip 2: Avoid tiny remote tasks
Ray APIs are general and simple to use. As a result, new comers' natural instinct is to parallelize all tasks, including tiny ones, which can incur an overhead overtime. 
In short, if the Ray remote tasks are tiny or miniscule in compute, they may take longer to execute than their serial Python equivalents.

In [12]:
def tiny_task(x):
    time.sleep(0.0001)
    return 2 * x

In [13]:
%%time
results = [tiny_task(x) for x in range(100000)]
results[:10]

CPU times: user 140 ms, sys: 185 ms, total: 325 ms
Wall time: 12.9 s


[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

Now convert this into Ray remote task

In [14]:
@ray.remote
def remote_tiny_task(x):
    time.sleep(0.0001)
    return x

In [15]:
%%time
result_ids = [remote_tiny_task.remote(x) for x in range(100000)]
results = ray.get(result_ids)
results[:10]

CPU times: user 7.29 s, sys: 3.18 s, total: 10.5 s
Wall time: 12 s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Surprisingly, Ray didn’t improve the execution time. Ray program is actually slower or closer in execution time than the sequential program! 

What's going on? What can we do to remedy it?

Well, the issue here is that every task invocation has a non-trivial overhead (e.g., scheduling, inter-process communication, updating the system state), and this overhead dominates the actual time it takes to execute the task.

One way to mitigate is to make the remote tasks "larger" in order to amortize invocation overhead. This is achieved by aggregating tasks into bigger chunks of 1000.


In [16]:
@ray.remote
def mega_work(start, end):
    return [tiny_task(x) for x in range(start, end)]

In [17]:
%%time
result_ids = []
[result_ids.append(mega_work.remote(x*1000, (x+1)*1000)) for x in range(100)]
results = ray.get(result_ids)

CPU times: user 219 ms, sys: 32.1 ms, total: 251 ms
Wall time: 4 s


A huge difference in execution time, almost **4X** faster!

### Tip 3: Using ray.wait() with ray.get()

As we noted above, an idiomatic way of using `ray.get()` is to delay fetching the object until you need them. Another way is to use it is with `ray.wait()`. Only fetch values that are already available or materialized in the object store. This is a way to [pipeline the execution](https://docs.ray.io/en/latest/ray-core/tips-for-first-time.html#tip-4-pipeline-data-processing), especially when you want to process the results of completed Ray tasks.

|<img src="https://docs.ray.io/en/latest/_images/pipeline.png" height="20%" width="40%">|
|:--|
|Execution timeline in both cases: when using `ray.get()` to wait for all results to become available before processing them, and using `ray.wait()` to start processing the results as soon as they become available.|


If we use `ray.get()` on the results of multiple tasks we will have to wait until the last one of these tasks finishes. This can be an issue if tasks take widely different amounts of time.

To illustrate this issue, consider the following example where we run four `transform_images()` tasks in parallel, with each task taking a time uniformly distributed between 0 and 4 seconds. Next, assume the results of these tasks are processed by `classify_images()`, which takes 1 sec per result. The expected running time is then (1) the time it takes to execute the slowest of the `transform_images()` tasks, plus (2) 4 seconds which is the time it takes to execute `classify_images()`.

Let's look at a simple example.

#### Not using ray.wait and no pipelining

In [7]:
from PIL import Image, ImageFilter
from torchvision import transforms as T

In [8]:
random.seed(42)

In [41]:
import time
import random
import ray

@ray.remote
def transform_images(x):
    imarray = np.random.rand(x, x , 3) * 255
    img = Image.fromarray(imarray.astype('uint8')).convert('RGBA')
    
    # Make the image blur with specified intensify
    img = img.filter(ImageFilter.GaussianBlur(radius=20))
    
    time.sleep(random.uniform(0, 4)) # Replace this with extra work you need to do.
    return img

def predict(image):
    size = image.size[0]
    if size == 16 or size == 32:
        return 0
    elif size == 64 or size == 128:
        return 1
    elif size == 256:
        return 2
    else:
        return 3

def classify_images(images):
    preds = []
    for image in images:
        pred = predict(image)
        time.sleep(1)
        preds.append(pred)
    return preds

def classify_images_inc(images):
    preds = [predict(img) for img in images]
    time.sleep(1)
    return preds

SIZES = [16, 32, 64, 128, 256, 512]

In [42]:
%%time

start = time.time()
images = ray.get([transform_images.remote(image) for image in SIZES])
predictions = classify_images(images)
print(f"Duration: {round(time.time() - start, 2)} seconds and predictions: {predictions}")

Duration: 9.54 seconds and predictions: [0, 0, 1, 1, 2, 3]
CPU times: user 64.8 ms, sys: 34.8 ms, total: 99.5 ms
Wall time: 9.54 s


#### Using ray.wait and pipelining

In [43]:
%%time

start = time.time()
result_images_refs = [transform_images.remote(image) for image in SIZES] 
predictions =[]

# Loop until all tasks are finished
while len(result_images_refs):
    done_image_refs, result_images_refs = ray.wait(result_images_refs, num_returns=1)
    preds = classify_images_inc(ray.get(done_image_refs))
    predictions.extend(preds)
print(f"Duration: {round(time.time() - start, 2)} seconds and predictions: {predictions}")

Duration: 6.96 seconds and predictions: [0, 1, 1, 2, 0, 3]
CPU times: user 52.6 ms, sys: 30.9 ms, total: 83.5 ms
Wall time: 6.96 s


**Notice**: You get some incremental difference. However, for compute intensive and many tasks, and overtime, this difference will be in order of magnitude.

### Tip 4: Avoid passing same object repeatedly to remote tasks¶

When you pass a large object as an argument to a remote function, Ray calls `ray.put()` under the hood to store that object in the local object store. Done once with the same object, say outside a loop, can significantly improve the performance of a remote task invocation when the remote task is executed locally, or if running on the same cluster node, as all tasks on the same node share the object store in shared memory. But if done by passing the same object repeatedly inside a loop can degrade the performance.

In [23]:
@ray.remote
def do_work(a):
    return np.sum(a)

In [24]:
start = time.time()
a = np.random.rand(5000, 5000)

# Sending the big array to each remote task
result_ids = [do_work.remote(a) for x in range(10)]
results = math.fsum(ray.get(result_ids))
print(f" results = {results:.2f} and duration = {time.time() - start:.3f} sec")

 results = 124992196.47 and duration = 0.666 sec


Now, we going to store the large array into the object store, so there is only a
single copy and added only once. Plus we sending not the array but a reference to it.

In [25]:
start = time.time()

# Adding the big array into the object store
a_id = ray.put(np.random.rand(5000, 5000))
result_ids = [do_work.remote(a_id) for x in range(10)]
results = math.fsum(ray.get(result_ids))
print(f" results = {results:.2f} and duration = {time.time() - start:.3f} sec")

 results = 125007941.16 and duration = 0.380 sec


### Exercise

For **Tip 3**:
 * Extend two more images of sizes: 1024, 2048
 * Increase the number of returns to 2 from the `ray.wait`()`
 * Process two images at a time
 * Note the difference in processing time

In [44]:
ray.shutdown()

### Summary

In this short tutorial, we got a short glimpse at the Ray Core APIs. By no means it was comprehensive, but we touched on some methods we 
have seen in the previous lessons; however, here with those methods, we explored additional arguments to the `.remote()` call such as number of return
statements as well as how to supply runtime environments and dependencies for your Ray cluster during `ray.init()` call. Note that some arguments to `ray.init()` 
can also be supplied to `ray.remote()` decorator, such as num_cpus, num_gpus, runtime_env, etc. 

More importantly, we walked through some tips and tricks that many developers new to Ray can easily stumble upon. Although the examples were short and simple,
the idea and cautionary tales are important part of the learning process.

### Next Step

Let's move on to our final module 3 and start with [Ray Datasets lesson](ex_07_ray_data.ipynb)

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_07_ray_data.ipynb) <br>
⬅️ [Previous notebook](./ex_05_multiprocess_pool.ipynb) <br>